In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib
import torch
import random
import unidecode
import re
from datetime import datetime

In [13]:
df = pd.read_excel('data/DataFinal/model/PHIẾU TRƯNG CẦU Ý KIẾN NGƯỜI ĐI LÀM (Câu trả lời).xlsx', header=0)
df = df.drop(index=0)

# Mô tả dữ liệu

In [14]:
# 1. Tổng quan về dữ liệu
print("Tổng số dòng và cột trong dữ liệu:")
print(df.shape)

# print("\nCác thuộc tính trong dữ liệu:")
# print(df.columns)

df.info()


Tổng số dòng và cột trong dữ liệu:
(1067, 307)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 1 to 1067
Columns: 307 entries, Dấu thời gian to Câu 12. Tích vào những câu trả lời sau đây theo từng mức độ mà Anh/chị cho là đúng với mình.  [Tôi đang phát huy hết năng lực mình có trong công việc].1
dtypes: datetime64[ns](1), float64(153), int64(6), object(147)
memory usage: 2.5+ MB


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 1 to 1067
Columns: 307 entries, Dấu thời gian to Câu 12. Tích vào những câu trả lời sau đây theo từng mức độ mà Anh/chị cho là đúng với mình.  [Tôi đang phát huy hết năng lực mình có trong công việc].1
dtypes: datetime64[ns](1), float64(153), int64(6), object(147)
memory usage: 2.5+ MB


In [16]:
# In danh sách tên các cột
print(df.columns.tolist())

['Dấu thời gian', 'Phần 1. NHỮNG THÔNG TIN CHUNG', 'Câu 1.  Anh/chị đã học trường THCS nào (Tên trường - Huyện -  Tỉnh)?', 'Câu 2.   Loại hình trường khi anh/chị học THCS?', 'Câu 3.   Loại hình lớp khi anh/chị học THCS?', 'Câu 4.    Nguyên quán/nơi sinh/nơi thường trú (khi anh/chị học THCS)  ?', 'Câu 5.    Giới tính ?', 'Câu 6.    Dân tộc ?', 'Câu 7.   Tuổi của anh/chị ?', 'Câu 8.    Sau khi học xong THCS thì anh/chị?', 'Câu 8a.    Nghề nghiệp của anh/chị sau khi học xong THCS/THPT/Trung cấp/Cao đẳng/Đại học? ', 'Câu 9.    Nghề nghiệp hiện tại của anh/chị ? ', 'Câu 10.     Anh/chị đã từng thay đổi công việc?  ', 'Câu 10a.     Nếu có, Lý do Anh/chị đã từng thay đổi công việc?  ', 'Câu 10b.     Nêu Lý do khác khiến Anh/chị thay đổi công việc?  ', 'Câu 10c.      Công việc mà anh/chị cho là thành công nhất nhờ việc hướng nghiệp ở THCS?', 'Câu 11.      Nghề nghiệp mà anh/chị cho là thành công nhất nhờ việc hướng nghiệp ở THCS?', 'Câu 11a. Tên cụ thể nghề khác?', 'Câu 12.       Nghề nghiệp c

In [17]:
# Đổi tên cột
def rename_columns(original_columns):
    new_columns = []
    rename_dict = {}
    column_count = {}  
    
    for col in original_columns:
        cleaned_name = unidecode.unidecode(col.strip())
        cleaned_name = ''.join(e if e.isalnum() or e in ['_'] else '_' for e in cleaned_name)
        short_name = cleaned_name[:30]  
    
        if short_name in column_count:
            column_count[short_name] += 1
            short_name = f"{short_name}_{column_count[short_name]}" 
        else:
            column_count[short_name] = 1
        
        new_columns.append(short_name)
        rename_dict[col] = short_name

    return new_columns, rename_dict

In [18]:
# Gọi hàm để đổi tên cột
new_column_names, rename_mapping = rename_columns(df.columns.tolist())

# Đặt tên cột mới cho DataFrame
df.columns = new_column_names

# Lưu lại mapping vào file
rename_df = pd.DataFrame(list(rename_mapping.items()), columns=['Tên cũ', 'Tên mới'])
rename_df.to_csv('model/SIU/rename_mapping.csv', index=False)

In [19]:
# In danh sách tên các cột
print(df.columns.tolist())

['Dau_thoi_gian', 'Phan_1__NHUNG_THONG_TIN_CHUNG', 'Cau_1___Anh_chi_da_hoc_truong_', 'Cau_2____Loai_hinh_truong_khi_', 'Cau_3____Loai_hinh_lop_khi_anh', 'Cau_4_____Nguyen_quan_noi_sinh', 'Cau_5_____Gioi_tinh__', 'Cau_6_____Dan_toc__', 'Cau_7____Tuoi_cua_anh_chi__', 'Cau_8_____Sau_khi_hoc_xong_THC', 'Cau_8a_____Nghe_nghiep_cua_anh', 'Cau_9_____Nghe_nghiep_hien_tai', 'Cau_10______Anh_chi_da_tung_th', 'Cau_10a______Neu_co__Ly_do_Anh', 'Cau_10b______Neu_Ly_do_khac_kh', 'Cau_10c_______Cong_viec_ma_anh', 'Cau_11_______Nghe_nghiep_ma_an', 'Cau_11a__Ten_cu_the_nghe_khac_', 'Cau_12________Nghe_nghiep_cua_', 'Cau_12a__Ten_cu_the_nghe_khac_', 'Cau_13________Nghe_nghiep_cua_', 'Cau_13a__Ten_cu_the_nghe_khac_', 'Cau_14________Diem_trung_binh_', 'Cau_15________Nam_hoc_lop_9_an', 'Cau_16________Nam_hoc_lop_9_an', 'Cau_17________Nam_hoc_lop_9_an', 'Cau_18________Nam_hoc_lop_9_an', 'Cau_1__Khi_hoc_THCS__anh_chi_c', 'Cau_1a___Neu_co__anh_chi_duoc_', 'Cau_1b__Nghe_khac_', 'Cau_1c__Anh_chi_dat_duoc_chung'

In [20]:
df = df.drop(columns=['Dau_thoi_gian', 'Phan_1__NHUNG_THONG_TIN_CHUNG', 
                      'Phan_1__NHUNG_THONG_TIN_CHUNG_', 'Cau_1___Anh_chi_da_hoc_truong__2', 'Cau_2____Loai_hinh_truong_khi__2', 'Cau_3____Loai_hinh_lop_khi_anh_2', 'Cau_4_____Nguyen_quan_noi_sinh_2', 'Cau_5_____Gioi_tinh___1', 'Cau_6_____Dan_toc___1', 'Cau_7____Tuoi_cua_anh_chi___1', 'Cau_8_____Sau_khi_hoc_xong_THC_2', 'Cau_8a_____Nghe_nghiep_cua_anh_2', 'Cau_9_____Nghe_nghiep_hien_tai_2', 'Cau_10______Anh_chi_da_tung_th_2', 'Cau_10a______Neu_co__Ly_do_Anh_2', 'Cau_10b______Neu_Ly_do_khac_kh_2', 'Cau_10c_______Cong_viec_ma_anh_2', 'Cau_11_______Nghe_nghiep_ma_an_2', 'Cau_11a__Ten_cu_the_nghe_khac__2', 'Cau_12________Nghe_nghiep_cua__2', 'Cau_12a__Ten_cu_the_nghe_khac__2', 'Cau_13________Nghe_nghiep_cua__2', 'Cau_13a__Ten_cu_the_nghe_khac__2', 'Cau_14________Diem_trung_binh__2', 'Cau_15________Nam_hoc_lop_9_an_2', 'Cau_16________Nam_hoc_lop_9_an_2', 'Cau_17________Nam_hoc_lop_9_an_2', 'Cau_18________Nam_hoc_lop_9_an_2', 'Cau_1__Khi_hoc_THCS__anh_chi_c_2', 'Cau_1a___Neu_co__anh_chi_duoc__2', 'Cau_1b__Nghe_khac__1', 'Cau_1c__Anh_chi_dat_duoc_chung_2', 'Cau_1d___Cong_viec_hien_nay_cu_2', 'Cau_1e____Trong_cuoc_song__anh_2', 'Cau_2__Khi_hoc_THCS__anh_chi_d_7', 'Cau_2__Khi_hoc_THCS__anh_chi_d_8', 'Cau_2__Khi_hoc_THCS__anh_chi_d_9', 'Cau_2__Khi_hoc_THCS__anh_chi_d_10', 'Cau_2__Khi_hoc_THCS__anh_chi_d_11', 'Cau_2__Khi_hoc_THCS__anh_chi_d_12', 'Cau_3___Khi_hoc_THCS__anh_chi__9', 'Cau_3___Khi_hoc_THCS__anh_chi__10', 'Cau_3___Khi_hoc_THCS__anh_chi__11', 'Cau_3___Khi_hoc_THCS__anh_chi__12', 'Cau_3___Khi_hoc_THCS__anh_chi__13', 'Cau_3___Khi_hoc_THCS__anh_chi__14', 'Cau_3___Khi_hoc_THCS__anh_chi__15', 'Cau_3___Khi_hoc_THCS__anh_chi__16', 'Cau_3a___Ghi_ro_co_so_khac_1', 'Cau_4__Khi_hoc_THCS__anh_chi_d_12', 'Cau_4__Khi_hoc_THCS__anh_chi_d_13', 'Cau_4__Khi_hoc_THCS__anh_chi_d_14', 'Cau_4__Khi_hoc_THCS__anh_chi_d_15', 'Cau_4__Khi_hoc_THCS__anh_chi_d_16', 'Cau_4__Khi_hoc_THCS__anh_chi_d_17', 'Cau_4__Khi_hoc_THCS__anh_chi_d_18', 'Cau_4__Khi_hoc_THCS__anh_chi_d_19', 'Cau_4__Khi_hoc_THCS__anh_chi_d_20', 'Cau_4__Khi_hoc_THCS__anh_chi_d_21', 'Cau_4__Khi_hoc_THCS__anh_chi_d_22', 'Cau_4a___Ghi_ro_hoat_dong_khac_2', 'Cau_5__Khi_hoc_THCS__anh_chi_t_22', 'Cau_5__Khi_hoc_THCS__anh_chi_t_23', 'Cau_5__Khi_hoc_THCS__anh_chi_t_24', 'Cau_5__Khi_hoc_THCS__anh_chi_t_25', 'Cau_5__Khi_hoc_THCS__anh_chi_t_26', 'Cau_5__Khi_hoc_THCS__anh_chi_t_27', 'Cau_5__Khi_hoc_THCS__anh_chi_t_28', 'Cau_5__Khi_hoc_THCS__anh_chi_t_29', 'Cau_5__Khi_hoc_THCS__anh_chi_t_30', 'Cau_5__Khi_hoc_THCS__anh_chi_t_31', 'Cau_5__Khi_hoc_THCS__anh_chi_t_32', 'Cau_5__Khi_hoc_THCS__anh_chi_t_33', 'Cau_5__Khi_hoc_THCS__anh_chi_t_34', 'Cau_5__Khi_hoc_THCS__anh_chi_t_35', 'Cau_5__Khi_hoc_THCS__anh_chi_t_36', 'Cau_5__Khi_hoc_THCS__anh_chi_t_37', 'Cau_5__Khi_hoc_THCS__anh_chi_t_38', 'Cau_5__Khi_hoc_THCS__anh_chi_t_39', 'Cau_5__Khi_hoc_THCS__anh_chi_t_40', 'Cau_5__Khi_hoc_THCS__anh_chi_t_41', 'Cau_5__Khi_hoc_THCS__anh_chi_t_42', 'Cau_5a___Ghi_ro_yeu_to_khac_1', 'Cau_6__Khi_hoc_THCS__anh_chi_t_7', 'Cau_6__Khi_hoc_THCS__anh_chi_t_8', 'Cau_6__Khi_hoc_THCS__anh_chi_t_9', 'Cau_6__Khi_hoc_THCS__anh_chi_t_10', 'Cau_6__Khi_hoc_THCS__anh_chi_t_11', 'Cau_6__Khi_hoc_THCS__anh_chi_t_12', 'Cau_7__Khi_hoc_THCS__anh_chi_b_19', 'Cau_7__Khi_hoc_THCS__anh_chi_b_20', 'Cau_7__Khi_hoc_THCS__anh_chi_b_21', 'Cau_7__Khi_hoc_THCS__anh_chi_b_22', 'Cau_7__Khi_hoc_THCS__anh_chi_b_23', 'Cau_7__Khi_hoc_THCS__anh_chi_b_24', 'Cau_7__Khi_hoc_THCS__anh_chi_b_25', 'Cau_7__Khi_hoc_THCS__anh_chi_b_26', 'Cau_7__Khi_hoc_THCS__anh_chi_b_27', 'Cau_7__Khi_hoc_THCS__anh_chi_b_28', 'Cau_7__Khi_hoc_THCS__anh_chi_b_29', 'Cau_7__Khi_hoc_THCS__anh_chi_b_30', 'Cau_7__Khi_hoc_THCS__anh_chi_b_31', 'Cau_7__Khi_hoc_THCS__anh_chi_b_32', 'Cau_7__Khi_hoc_THCS__anh_chi_b_33', 'Cau_7__Khi_hoc_THCS__anh_chi_b_34', 'Cau_7__Khi_hoc_THCS__anh_chi_b_35', 'Cau_7__Khi_hoc_THCS__anh_chi_b_36', 'Cau_8__Khi_hoc_THCS__anh_chi_c_2', 'Cau_9__Khi_hoc_THCS__anh_chi_c_2', 'Cau_10__Hay_cho_biet_vi_sao_An_28', 'Cau_10__Hay_cho_biet_vi_sao_An_29', 'Cau_10__Hay_cho_biet_vi_sao_An_30', 'Cau_10__Hay_cho_biet_vi_sao_An_31', 'Cau_10__Hay_cho_biet_vi_sao_An_32', 'Cau_10__Hay_cho_biet_vi_sao_An_33', 'Cau_10__Hay_cho_biet_vi_sao_An_34', 'Cau_10__Hay_cho_biet_vi_sao_An_35', 'Cau_10__Hay_cho_biet_vi_sao_An_36', 'Cau_10__Hay_cho_biet_vi_sao_An_37', 'Cau_10__Hay_cho_biet_vi_sao_An_38', 'Cau_10__Hay_cho_biet_vi_sao_An_39', 'Cau_10__Hay_cho_biet_vi_sao_An_40', 'Cau_10__Hay_cho_biet_vi_sao_An_41', 'Cau_10__Hay_cho_biet_vi_sao_An_42', 'Cau_10__Hay_cho_biet_vi_sao_An_43', 'Cau_10__Hay_cho_biet_vi_sao_An_44', 'Cau_10__Hay_cho_biet_vi_sao_An_45', 'Cau_10__Hay_cho_biet_vi_sao_An_46', 'Cau_10__Hay_cho_biet_vi_sao_An_47', 'Cau_10__Hay_cho_biet_vi_sao_An_48', 'Cau_10__Hay_cho_biet_vi_sao_An_49', 'Cau_10__Hay_cho_biet_vi_sao_An_50', 'Cau_10__Hay_cho_biet_vi_sao_An_51', 'Cau_10__Hay_cho_biet_vi_sao_An_52', 'Cau_10__Hay_cho_biet_vi_sao_An_53', 'Cau_10__Hay_cho_biet_vi_sao_An_54', 'Cau_11__Hien_tai__Anh_chi_cam__5', 'Cau_11__Hien_tai__Anh_chi_cam__6', 'Cau_11__Hien_tai__Anh_chi_cam__7', 'Cau_11__Hien_tai__Anh_chi_cam__8', 'Cau_12__Tich_vao_nhung_cau_tra_16', 'Cau_12__Tich_vao_nhung_cau_tra_17', 'Cau_12__Tich_vao_nhung_cau_tra_18', 'Cau_12__Tich_vao_nhung_cau_tra_19', 'Cau_12__Tich_vao_nhung_cau_tra_20', 'Cau_12__Tich_vao_nhung_cau_tra_21', 'Cau_12__Tich_vao_nhung_cau_tra_22', 'Cau_12__Tich_vao_nhung_cau_tra_23', 'Cau_12__Tich_vao_nhung_cau_tra_24', 'Cau_12__Tich_vao_nhung_cau_tra_25', 'Cau_12__Tich_vao_nhung_cau_tra_26', 'Cau_12__Tich_vao_nhung_cau_tra_27', 'Cau_12__Tich_vao_nhung_cau_tra_28', 
                      'Cau_12__Tich_vao_nhung_cau_tra_29', 'Cau_12__Tich_vao_nhung_cau_tra_30'], errors='ignore')

In [21]:
def remove_rows_with_high_nan(df, threshold=0.5):
    # Thay thế các giá trị 'None' và 'nan' bằng np.nan
    df.replace(['None', 'nan', 'NaN', ' ', ''], np.nan, inplace=True)
    
    # Tính tỷ lệ NaN trên mỗi hàng
    nan_ratio = df.isna().mean(axis=1)
    
    # Lọc các hàng có tỷ lệ NaN lớn hơn hoặc bằng ngưỡng
    filtered_df = df[nan_ratio < threshold]
    
    # Đếm số lượng các hàng bị loại bỏ (tức là có tỷ lệ NaN >= threshold)
    count_high_nan_rows = (nan_ratio >= threshold).sum()
    
    return filtered_df, count_high_nan_rows

In [22]:
df_cleaned, num_removed = remove_rows_with_high_nan(df, threshold=0.5)
print(f"Số lượng dòng bị loại bỏ: {num_removed}")

Số lượng dòng bị loại bỏ: 0


# Hàm

In [23]:
def calculate_percentage(df, column_name):
    # Tính số lượng xuất hiện của mỗi giá trị
    value_counts = df[column_name].value_counts(dropna=False)
    
    # Tính tỷ lệ phần trăm
    percentages = (value_counts / len(df)) * 100
    
    # Tạo DataFrame kết quả
    result = pd.DataFrame({'Value': value_counts.index, 'Percentage': percentages.values})
    
    return result

In [24]:
def calculate_percentage_for_columns(df, columns):
    # Khởi tạo dictionary để lưu kết quả của từng cột
    result_dict = {}
    
    # Duyệt qua từng cột trong danh sách và tính tỷ lệ phần trăm
    for column in columns:
        try:
            result_dict[column] = calculate_percentage(df, column)
        except KeyError:
            print(f"Cột '{column}' không tồn tại trong DataFrame")
    
    # In kết quả
    for column, result_df in result_dict.items():
        print(f"Tỷ lệ phần trăm xuất hiện trong cột '{column}':")
        print(result_df)
        print("\n")  # Thêm khoảng trắng giữa các kết quả cho dễ đọc
    
    # Trả về dictionary chứa các kết quả
    return result_dict

In [25]:
def show_infor_dataframe(df):
    for column in df.columns:
        unique_values = df[column].unique()
        print(f"Column '{column}' has {len(unique_values)} unique values:")
        print("\n")

In [26]:
def show_infor_dataframe_unique(df):
    for column in df.columns:
        unique_values = df[column].unique()
        print(f"Column '{column}' has {len(unique_values)} unique values:")
        print(unique_values)
        print("\n")

In [27]:
def display_column_info(df):
    for column in df.columns:
        # Kiểu dữ liệu của cột
        col_type = df[column].dtype
        
        # Số lượng giá trị không bị thiếu
        non_null_count = df[column].notna().sum()
        
        # Tổng số giá trị
        total_count = len(df[column])
        
        # Tính số lượng và tỷ lệ phần trăm giá trị thiếu
        missing_count = total_count - non_null_count
        missing_percentage = (missing_count / total_count) * 100
        
        
        print(f"Column: {column}")
        print(f"  Data Type: {col_type}")
        print(f"  Non-null Count: {non_null_count}")
        print(f"  Total Count: {total_count}")
        print(f"  Missing Values Count: {missing_count}")
        print(f"  Missing Values Percentage: {missing_percentage:.2f}%")
        print()
        # Kiểm tra xem tỷ lệ giá trị thiếu có cao không
        if missing_percentage >= 1:
            print(f"  Column {column} có {missing_percentage:.2f}% giá trị bị thiếu , cần xử lý")
        print()
        print()

# Dữ liệu thiếu

In [28]:
display_column_info(df)

Column: Cau_1___Anh_chi_da_hoc_truong_
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_2____Loai_hinh_truong_khi_
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_3____Loai_hinh_lop_khi_anh
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_4_____Nguyen_quan_noi_sinh
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_5_____Gioi_tinh__
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_6_____Dan_toc__
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_7____Tuoi_cu

# Dữ liệu trong cột

In [29]:
show_infor_dataframe(df)

Column 'Cau_1___Anh_chi_da_hoc_truong_' has 783 unique values:


Column 'Cau_2____Loai_hinh_truong_khi_' has 3 unique values:


Column 'Cau_3____Loai_hinh_lop_khi_anh' has 2 unique values:


Column 'Cau_4_____Nguyen_quan_noi_sinh' has 749 unique values:


Column 'Cau_5_____Gioi_tinh__' has 2 unique values:


Column 'Cau_6_____Dan_toc__' has 2 unique values:


Column 'Cau_7____Tuoi_cua_anh_chi__' has 76 unique values:


Column 'Cau_8_____Sau_khi_hoc_xong_THC' has 3 unique values:


Column 'Cau_8a_____Nghe_nghiep_cua_anh' has 6 unique values:


Column 'Cau_9_____Nghe_nghiep_hien_tai' has 6 unique values:


Column 'Cau_10______Anh_chi_da_tung_th' has 2 unique values:


Column 'Cau_10a______Neu_co__Ly_do_Anh' has 38 unique values:


Column 'Cau_10b______Neu_Ly_do_khac_kh' has 112 unique values:


Column 'Cau_10c_______Cong_viec_ma_anh' has 333 unique values:


Column 'Cau_11_______Nghe_nghiep_ma_an' has 8 unique values:


Column 'Cau_11a__Ten_cu_the_nghe_khac_' has 86 unique values:


Colu

In [30]:
show_infor_dataframe_unique(df)

Column 'Cau_1___Anh_chi_da_hoc_truong_' has 783 unique values:
['THCS Cộng Hoà'
 'THCS Chuyên Hoài Đức (nay là THCS Nguyễn Văn Huyên), Huyện Hoài Đức, Hà Tây'
 'Trường THPT Võ Thành Trinh' 'NGUYỄN VĂN TÂY, CHỢ MỚI, AG'
 'THCS Trần Huy Liệu- Vụ Bản- Nam Định' 'hải hậu c'
 'Mai Động - quận Hoàng Mai - Hà nội' 'chuyên Thoại Ngọc Hầu'
 'THCS Mỹ An - Chợ Mới - An Giang '
 'THCS AN THẠNH TRUNG - CHỢ MỚI - AN GIANG'
 'Thcs Trần Hưng Đạo - TP Long Xuyên - An Giang'
 'Giảng Võ, Ba Đình, Hà Nội '
 'THCS Bình Phước Xuân,huyện Chợ Mới tỉnh An giang'
 'Trương THCS Thị Trấn Núi sập ' 'Vũ Vinh - Vũ Thư - Thái Bình'
 'THCS Bình Mỹ- Châu Phú- An Giang'
 'Trường THCS Nguyễn Văn Huyên - Hoài Đức - Hà Nội'
 'THCS SƠN ĐÀ - BA VÌ - HA NỘI' 'THCS TÒNG BẠT-BA VÌ - HÀ NỘI'
 'THCS Vân Hoà, Ba Vì, Hà Nội ' 'THCS Vạn Phúc - Quận Hà Đông - Hà Nội'
 'Tân Tiến, huyện Văn Giang, tỉnh Hưng Yên '
 'THCS Ba Trại - huyện Ba Vì - Hà Nội'
 'Nguyễn Văn Huyên - Hoài Đức- Hà Nội' 'Cẩm Lĩnh - Ba Vì - Hà Nội'
 'Trường THCS Chu 

Có quá nhiều dữ liệu khác nhau, không thể chuẩn hóa được:
Column 'Cau_1___Anh_chi_da_hoc_truong_' has 783 unique values
Column 'Cau_4_____Nguyen_quan_noi_sinh' has 749 unique values

Có quá nhiều dữ liệu khác nhau, cần quan sát đặc trưng và chuẩn hóa kĩ:
Column 'Cau_7____Tuoi_cua_anh_chi__' has 76 unique values
Column 'Cau_10a______Neu_co__Ly_do_Anh' has 38 unique values
Column 'Cau_10b______Neu_Ly_do_khac_kh' has 112 unique values
Column 'Cau_10c_______Cong_viec_ma_anh' has 333 unique values
Column 'Cau_11a__Ten_cu_the_nghe_khac_' has 86 unique values
Column 'Cau_12a__Ten_cu_the_nghe_khac_' has 112 unique values
Column 'Cau_13a__Ten_cu_the_nghe_khac_' has 97 unique values
Column 'Cau_1b__Nghe_khac_' has 87 unique values
Column 'Cau_4a___Ghi_ro_hoat_dong_khac' has 43 unique values
Column 'Cau_5a___Ghi_ro_yeu_to_khac' has 64 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b' has 41 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_2' has 44 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_3' has 44 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_4' has 44 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_5' has 46 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_6' has 41 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_7' has 37 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_8' has 29 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_9' has 29 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_10' has 43 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_11' has 45 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_12' has 45 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_13' has 46 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_14' has 41 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_15' has 42 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_16' has 45 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_17' has 48 unique values
Column 'Cau_7__Khi_hoc_THCS__anh_chi_b_18' has 53 unique values

Có lỗi cần xử lý riêng từng trường hơp

Column 'Cau_12________Nghe_nghiep_cua_' has 9 unique values:
['Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất…)'
 'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất, nông nghiệp, thủ công…)'
 'Nhóm nghề khác'
 'Quản lý (tiếp thị, bán hàng, quản trị kinh doanh, cảnh sát, thanh tra,…)'
 'Xã hội (giáo viên, huấn luyện viên, hoạt động công tác xã hội…)'
 'Nghiệp vụ (như thư kí, kiểm toán, kế toán, hành chính văn phòng…)'
 'Nghiên cứu (như toán học, tâm lí học, xã hội học, Khoa học tự nhiên,…)'
 'Nghệ thuật (viết văn, đạo diễn, dẫn chương trình, diễn viên, họa sĩ,…)'
 'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất, nông nghiệp, thủ công…)']


Column 'Cau_1a___Neu_co__anh_chi_duoc_' has 10 unique values:
['Kĩ thuật, công nghệ (như điện, kĩ thuật nông nghiệp, xây dựng, cơ khí, điện tử, địa chất…)'
 'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất, nông nghiệp, thủ công…)'
 nan 'Xã hội (giáo viên, huấn luyện viên, hoạt động công tác xã hội…)'
 'Nhóm nghề khác'
 'Nghệ thuật (viết văn, đạo diễn, dẫn chương trình, diễn viên, họa sĩ,…)'
 'Quản lý (tiếp thị, bán hàng, quản trị kinh doanh, cảnh sát, thanh tra,…)'
 'Nghiên cứu (như toán học, tâm lí học, xã hội học, Khoa học tự nhiên,…)'
 'Nghiệp vụ (như thư kí, kiểm toán, kế toán, hành chính văn phòng…)'
 'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất, nông nghiệp, thủ công…)']

Column 'Cau_5__Khi_hoc_THCS__anh_chi_t' has 8 unique values:
['Ảnh hưởng mạnh' 'Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Không ảnh hưởng' 'Ảnh hưởng đôi chút' 'Ảnh hưởng mạnh'
 'Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_2' has 9 unique values:
['Ảnh hưởng mạnh' 'Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Không ảnh hưởng' 'Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng' 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_3' has 8 unique values:
['Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng' 'Không ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Ảnh hưởng mạnh' 'Ảnh hưởng mạnh'
 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_4' has 9 unique values:
['Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng'
 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_5' has 8 unique values:
['Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_6' has 9 unique values:
['Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng'
 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_7' has 9 unique values:
['Ảnh hưởng mạnh' 'Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Không ảnh hưởng' 'Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng' 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_8' has 7 unique values:
['Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_9' has 7 unique values:
['Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_10' has 7 unique values:
['Ảnh hưởng mạnh' 'Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Tương đối ảnh hưởng'
 'Ảnh hưởng đôi chút']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_11' has 8 unique values:
['Ảnh hưởng mạnh' 'Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Tương đối ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_12' has 9 unique values:
['Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng'
 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_13' has 8 unique values:
['Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút' 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_14' has 7 unique values:
['Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_15' has 8 unique values:
['Ảnh hưởng mạnh' 'Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Không ảnh hưởng' 'Tương đối ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_16' has 8 unique values:
['Ảnh hưởng mạnh' 'Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Không ảnh hưởng' 'Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_17' has 8 unique values:
['Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút' 'Hầu như không ảnh hưởng']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_18' has 8 unique values:
['Tương đối ảnh hưởng' 'Hầu như không ảnh hưởng' 'Không ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Ảnh hưởng mạnh' 'Tương đối ảnh hưởng'
 'Ảnh hưởng đôi chút' 'Ảnh hưởng mạnh']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_19' has 7 unique values:
['Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_20' has 7 unique values:
['Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng' 'Ảnh hưởng đôi chút']


Column 'Cau_5__Khi_hoc_THCS__anh_chi_t_21' has 6 unique values:
['Ảnh hưởng đôi chút' 'Tương đối ảnh hưởng'
 'Hầu như không ảnh hưởng' 'Không ảnh hưởng' 'Ảnh hưởng mạnh'
 'Tương đối ảnh hưởng']

Có dữ liệu bị thiếu cần xử lý:
Cau_10a______Neu_co__Ly_do_Anh
Cau_10b______Neu_Ly_do_khac_kh
Cau_11a__Ten_cu_the_nghe_khac_
Cau_12a__Ten_cu_the_nghe_khac_
Cau_13a__Ten_cu_the_nghe_khac_
Cau_1a___Neu_co__anh_chi_duoc_
Cau_1b__Nghe_khac_
Cau_1c__Anh_chi_dat_duoc_chung
Cau_1d___Cong_viec_hien_nay_cu
Cau_1e____Trong_cuoc_song__anh
Cau_3a___Ghi_ro_co_so_khac
Cau_4a___Ghi_ro_hoat_dong_khac
Cau_5a___Ghi_ro_yeu_to_khac


In [31]:
df = df.drop(columns=['Cau_1___Anh_chi_da_hoc_truong_', 'Cau_4_____Nguyen_quan_noi_sinh'], errors='ignore')

In [32]:
def normalize_age(age):
    current_year = datetime.now().year  # Lấy năm hiện tại
    
    # Nếu là một giá trị kiểu số, trả về trực tiếp
    if isinstance(age, (int, float)):
        return age
    
    # 1. Chuyển đổi các chuỗi có dạng 'X tuổi' hoặc 'X tuổi '
    if isinstance(age, str):
        # Xóa khoảng trắng và kiểm tra các trường hợp 'X tuổi' hoặc 'X tuổi '
        age = age.strip()
        
        # Nếu có dạng 'X tuổi' hoặc 'X tuổi '
        match = re.match(r'^(\d+)\s?tuổi$', age)
        if match:
            return int(match.group(1))
        
        # Nếu có dạng 'X tuổi' với các khoảng trắng
        match = re.match(r'^(\d+)\s*tuổi\s*$', age)
        if match:
            return int(match.group(1))
        
        # 2. Nếu có dạng 'Sinh năm YYYY' thì tính tuổi
        match = re.match(r'^Sinh năm (\d{4})$', age)
        if match:
            birth_year = int(match.group(1))
            return current_year - birth_year
        
        # 3. Nếu là 'YYYY' thì tính tuổi từ năm sinh
        if age.isdigit() and len(age) == 4:
            birth_year = int(age)
            return current_year - birth_year
        
        # 4. Nếu có dạng 'Trên 50' thì gán là 51
        if 'Trên 50' in age:
            return 51
        
        # 5. Nếu có dạng '4y', có thể hiểu là '4 years old' => Trả về 4
        match = re.match(r'(\d+)y', age)
        if match:
            return 40
        
        # 6. Xử lý trường hợp có chuỗi chữ hoặc thông tin không hợp lệ
        if age.isdigit():
            return int(age)
        
        # 7. Nếu có dạng '12-15' thì gán là 14
        if '12-15' in age:
            return 14
        
    return np.nan  # Nếu không thể xử lý, trả về NaN

# Áp dụng hàm normalize_age lên cột 'Cau_7____Tuoi_cua_anh_chi__'
df['Cau_7____Tuoi_cua_anh_chi__'] = df['Cau_7____Tuoi_cua_anh_chi__'].apply(normalize_age)

# Xác định năm hiện tại
current_year = 2024

# Hàm tính tuổi và chuẩn hóa nhóm tuổi
def calculate_age_and_group(value):
    # Nếu là năm sinh
    if value >= 1900:
        age = current_year - value
    else:
        age = value

    # Nhóm tuổi
    if age <= 12:
        return "Độ tuổi từ 0 đến 12"
    elif 13 <= age <= 18:
        return "Độ tuổi từ 13 đến 18"
    elif 19 <= age <= 29:
        return "Độ tuổi từ 19 đến 29"
    elif 30 <= age <= 39:
        return "Độ tuổi từ 30 đến 39"
    elif 40 <= age <= 49:
        return "Độ tuổi từ 40 đến 49"
    else:
        return "Độ tuổi từ 50 trở lên"

# Áp dụng tính toán và chuẩn hóa
df["Cau_7____Tuoi_cua_anh_chi__"] = df["Cau_7____Tuoi_cua_anh_chi__"].apply(calculate_age_and_group)


In [33]:
from unidecode import unidecode
# Danh sách lý do đầy đủ
ly_do = [
    "Lí do khác", "Tính ổn định của công việc trong tương lai", "Công việc quá khó",
    "Đãi ngộ không công bằng", "Công việc không ưu thích", "Lượng công việc nhiều",
    "Chậm thăng tiến, thăng cấp", "Không được nghỉ vào ngày nghỉ", "Mẫu thuẫn với đồng nghiệp"
]

# Chuẩn hóa tên cột thành tiếng Việt không dấu và không có khoảng trắng
column_names = [
    f"Cau_10a_____{unidecode(reason).replace(' ', '_').replace(',', '').lower()}"
    for reason in ly_do
]

# Tạo các cột nhị phân
for reason, col_name in zip(ly_do, column_names):
    df[col_name] = df['Cau_10a______Neu_co__Ly_do_Anh'].apply(
        lambda x: 1 if pd.notna(x) and reason in x else 0
    )

# Tạo cột `cau_10a_____li_do_khac` và đặt giá trị mặc định là 0
if 'cau_10a_____li_do_khac' not in df.columns:
    df['cau_10a_____li_do_khac'] = 0

# Xử lý giá trị NaN: Nếu giá trị là NaN, điền 1 vào cột `cau_10a_____li_do_khac`
df['cau_10a_____li_do_khac'] = df.apply(
    lambda row: 1 if pd.isna(row['Cau_10a______Neu_co__Ly_do_Anh']) else row['cau_10a_____li_do_khac'],
    axis=1
)

df = df.drop(columns=['Cau_10a______Neu_co__Ly_do_Anh', 'cau_10a_____li_do_khac'], errors='ignore')


In [34]:

# # Chuẩn hóa dữ liệu
# def clean_column(value):
#     # 1. Chuyển thành chữ thường
#     if pd.isna(value):  # Kiểm tra giá trị NaN
#         return "Không thay đổi"
#     value = str(value).strip().lower()

#     # 2. Loại bỏ các ký tự không cần thiết
#     value = value.replace("  ", " ")  # Loại bỏ khoảng trắng thừa
#     value = ''.join(c for c in value if c.isalnum() or c.isspace())  # Chỉ giữ lại chữ cái, số và khoảng trắng

#     # 3. Chuẩn hóa giá trị trùng lặp
#     mapping = {
#         "khong": "không có lý do",
#         "ko": "không có lý do",
#         "không": "không có lý do",
#         "x": "không rõ",
#         "ề": "không rõ",
#         "xa nha": "thay đổi nơi ở",
#         "thu nhập thấp": "lương thấp",
#         "thay đổi môi trường công tác": "thay đổi môi trường",
#         "muốn thay đổi ngành nghề": "đổi ngành nghề",
#     }
#     return mapping.get(value, value)  # Trả về giá trị được ánh xạ, nếu không giữ nguyên

# # Áp dụng hàm chuẩn hóa
# df["Cau_10b______Neu_Ly_do_khac_kh"] = df["Cau_10b______Neu_Ly_do_khac_kh"].apply(clean_column)

# Tạo thêm cột nhóm lý do
def standardize_reasons(reason):
    if pd.isna(reason):
        return "Không thay đổi"
    reason = str(reason).strip().lower()
    if any(keyword in reason for keyword in ['lương', 'thu nhập']):
        return "Thu nhập không phù hợp"
    elif any(keyword in reason for keyword in ['môi trường', 'công việc không phù hợp']):
        return "Môi trường làm việc không phù hợp"
    elif any(keyword in reason for keyword in ['gia đình', 'ổn định']):
        return "Lý do gia đình"
    elif any(keyword in reason for keyword in ['thay đổi nơi ở', 'chuyển nơi ở']):
        return "Thay đổi chỗ ở"
    elif any(keyword in reason for keyword in ['ngành học', 'đam mê', 'công việc mình thích']):
        return "Theo đuổi đam mê/ngành học"
    elif reason in ['không', 'ko']:
        return "Không thay đổi"
    else:
        return "Lý do khác"


df["Cau_10b______Neu_Ly_do_khac_kh"] = df["Cau_10b______Neu_Ly_do_khac_kh"].apply(standardize_reasons)


In [35]:
# Định nghĩa các nhóm chính
grouping_dict = {
    'Giáo viên': ['Giáo Viên', 'GV', 'GIAO VIÊN', 'Giáo viên', 'Dạy học', 'Giảng dạy', 'Giáo viên Ngữ Văn'],
    'Kỹ thuật': ['kỹ thuật', 'Kỹ thuật', 'Kỹ thuật máy tính', 'Kỹ sư', 'Kỹ sư cơ khí chế tạo', 'Kỹ thuật công nghệ'],
    'Kinh doanh': ['Quản lí', 'Marketing', 'Kinh doanh', 'Kinh Doanh', 'Bán hàng', 'Công việc văn phòng'],
    'Hướng nghiệp': ['Chọn đúng nghề', 'Định hướng nghề nghiệp', 'Hướng nghiệp', 'Định hướng nghề nghiệp', 'Định hướng cho học sinh'],
    'Không có hướng nghiệp': ['Khi học THCS, tôi không được hướng nghiệp', 'Không biết', 'Không có', 'Chưa được hướng nghiệp', 'Không có hướng nghiệp'],
    'Khác': ['Xã hội', 'Trải nghiệm', 'Không rõ', 'Không có câu trả lời', 'Không nhớ', 'Không rõ được hướng nghiệp']
}

# Hàm chuẩn hóa theo nhóm
def normalize_job_category(value):
    for group, keywords in grouping_dict.items():
        if any(keyword.lower() in str(value).strip().lower() for keyword in keywords):
            return group
    return 'Khác'

# Áp dụng hàm chuẩn hóa vào DataFrame
df['Cau_10c_______Cong_viec_ma_anh'] = df['Cau_10c_______Cong_viec_ma_anh'].apply(normalize_job_category)


In [36]:

# Định nghĩa các nhóm nghề chính
job_groups = {
    'Giáo viên': ['Giáo viên', 'Giáo viên Toán', 'Giáo viên THPT', 'Giáo viên ', 'Giáo viên Toán', 'Giảng viên', 'GV', 'Giáo Viên '],
    'Kinh doanh': ['Kinh doanh', 'Kinh doanh ', 'Tự kinh doanh', 'Buôn bán', 'Quản trị kinh doanh', 'Bán Hàng'],
    'Kỹ thuật': ['Kỹ sư IT', 'Kỹ sư', 'Kỹ thuật điện cơ bản', 'Kỹ thuật viên máy tính', 'Kỹ sư điện máy', 'Cơ Khí - Điện Tử', 'Lập trình viên'],
    'Y tế': ['Điều dưỡng ', 'Nghề bếp ', 'Tư vấn tài chính', 'Y sinh học', 'Tư vấn hướng nghiệp', 'Biên tập viên', 'Hướng dẫn viên'],
    'Giám đốc / Quản lý': ['Giám đốc công ty', 'Quản lí', 'Quản trị kinh doanh'],
    'Khoa học / Nghiên cứu': ['Nghiên cứu', 'nghiên cứu khoa học tự nhiên'],
    'Quân đội': ['Quân đội', 'các ngành khối quân đội.'],
    'Văn phòng / Hành chính': ['Hành chính văn phòng', 'Kiểm toán', 'Kế toán'],
    'Nghề khác': ['.', 'Không biết ', 'Không có ý kiến', 'X', 'ko có hướng nghiệp thcd', 
                  'Ko có', 'Không hiểu câu hỏi', 'X', 'Học sinh', 'Mc', 'NVVP', 'Không có', 'không có'
                  'Không', 'Không có ', 'Không có ', 'Không hiểu câu hỏi', 'Chưa đi làm', 'Chưa thấy đổi', 'không có', 'Ko', 'Xã hội ', 'Huấn luyện viên', 'Sáng tạo ', 'Phóng viên truyền hình', 'Không có ý kiến', 'Thay đổi hướng phát triển nghề nghiệp '],
    'Nông nghiệp': ['Nông nghiệp', 'Nông dân'],
    'Du lịch': ['Du lịch'],
    'Mạng xã hội / Báo chí': ['Báo chí', 'Phóng viên truyền hình'],
}

# Hàm để chuẩn hóa các nghề
def group_job(job_name):
    for group, keywords in job_groups.items():
        if any(keyword.lower() in str(job_name).strip().lower() for keyword in keywords):
            return group
    return 'Nghề khác'  # Trả về nhóm khác nếu không tìm thấy nhóm nào khớp

# Áp dụng hàm chuẩn hóa vào cột 'Cau_11a__Ten_cu_the_nghe_khac_'
df['Cau_11a__Ten_cu_the_nghe_khac_'] = df['Cau_11a__Ten_cu_the_nghe_khac_'].apply(lambda x: group_job(str(x) if pd.notnull(x) else ''))


In [37]:
# Định nghĩa các nhóm nghề chính
job_groups = {
    'Nông nghiệp': ['Nông dân', 'Làm nông', 'Làm ruộng', 'Làm nông nghiệp', 'Nông nghiệp', 'Nông Dân', 'Nông dân nghèo', 
                    'Nghề làm nông', 'Nghề nông', 'Trồng trọt','Đánh bắt thủy hải sản ', 'Ngư dân '],
    'Kinh doanh': ['Kinh doanh', 'Buôn bán', 'Kinh doanh tự do', 'Kinh doanh tư nhân', 'Bán hàng', 'Bán Hàng ở chợ', 'Bán Hàng', 
                   'Tiểu thương', 'Buôn bán tự do', 'Buôn bán nhỏ', 'Buôn bán kinh doanh tự do mô hình nhỏ'],
    'Y tế': ['Bác sĩ', 'Bác sỹ', 'Y sĩ', 'Y sinh học', 'Bác sỹ - Dược sỹ'],
    'Giáo viên': ['Giáo viên', 'Giáo Viên', 'Giáo viên tiếng Anh', 'Giáo viên ', 'Giáo viên Toán', 'Giảng viên', 'GV'],
    'Công nhân': ['Công nhân', 'Công nhân bán xăng dầu', 'Công nhân ', 'Công nhân '],
    'Quân đội': ['Bộ đội', 'Bộ đội ', 'Bộ đội hưu trí ', 'Quân đội', 'Bộ đội', 'Bộ đội', 'Cảnh sát'],
    'Hành chính / Công chức': ['Hành chính', 'Công chức', 'Quản lý nhà nước', 'Ngành luật''Phó chủ tịch xã '],
    'Lao động tự do': ['Lao động tự do', 'Lao động phổ thông', 'Lao động Tự do', 'Lao động tự do', 'Nghề tự do ', 'Tự do', 'Nghề tự do', 
                       'LĐTD', 'Nghỉ Hưu','Công nhân lái xe','Tài xế', 'Xe ôm', 'Lái xe', 'Bảo vệ', 'Lái tàu hoả'],
    'Văn hóa / Báo chí': ['Báo chí', 'Đài truyền thang, truyền hinh'],
    'Giải trí / Nghệ thuật': ['Nghề bếp', 'Sửa chữa và kinh doanh đồng hồ', 'Hoạt động công tác xã hội', 'Thương nghiệp'],
    'Nghề khác đặc biệt': ['Bảo vệ', 'Thầu xây dựng', 'Sửa chữa và kinh doanh đồng hồ'],
    'Nghề khác': ['.', 'X', 'Không', 'Không có', 'Không ', 'Không có thông tin', 'Không có ý kiến', 'Không hiểu câu hỏi', 'Ko', 
                  'Không có', 'Chưa thấy đổi', 'Ko', 'Không có thông tin', 'X', 'Không biết', 'Thương binh'],
}

# Hàm để chuẩn hóa các nghề
def group_job(job_name):
    for group, keywords in job_groups.items():
        if any(keyword.lower() in str(job_name).strip().lower() for keyword in keywords):
            return group
    return 'Nghề khác'  # Trả về nhóm khác nếu không tìm thấy nhóm nào khớp

# Áp dụng hàm chuẩn hóa vào cột 'Cau_12a__Ten_cu_the_nghe_khac_'
df['Cau_12a__Ten_cu_the_nghe_khac_'] = df['Cau_12a__Ten_cu_the_nghe_khac_'].apply(lambda x: group_job(str(x) if pd.notnull(x) else ''))


In [38]:
# Định nghĩa các nhóm nghề chính
job_groups = {
    'Nông nghiệp': ['Nông dân', 'Làm nông', 'Làm ruộng', 'Nông nghiệp', 'Trồng trọt', 'Trồng trọt chăn nuôi', 
                    'Làm nông nghiệp', 'Nông dân nghèo', 'Nghề làm nông', 'Bần nông', 'Nông ', 'Nông nghiệp ', 
                    'Làm ruỗng', 'Trồng trọt chăn nuôi', 'Nông Dâ '],
    'Kinh doanh': ['Kinh doanh', 'Buôn bán', 'Buôn bán nhỏ', 'Kinh doanh tự do', 'Tiểu thương', 'Bán hàng', 'Bán quán', 'Buôn bán nhỏ lẻ', 'Buôn bán thu mua Hải Sản', 'bán hàng tự do', 'buôn bán nhỏ'],
    'Y tế': ['Y tá', 'Y sĩ', 'Điều dưỡng', 'Bác sĩ', 'Bác sĩ thú y', 'Bác sỹ'],
    'Giáo viên': ['Giáo viên', 'Giáo Viên', 'Chuyên viên văn phòng', 'Nhân viên văn phòng', 'Văn thư'],
    'Công nhân': ['Công nhân', 'Công nhân cấp dưỡng', 'Công nhân nhà máy Dệt', 'Công nhân thủy lợi', 'Công nhân hưu trí', 'Lao động phổ thông', 'Lao động phổ thông '],
    'Quân đội': ['Bộ đội', 'bộ đội', 'cảnh sát'],
    'Hành chính / Công chức': ['Hành chính', 'Cán bộ nhà nước', 'Nhân viên Đài truyền thanh', 'Đài truyền thanh, truyền hình'],
    'Lao động tự do': ['Tự do', 'Lao động tự do', 'LĐTD', 'lao động phổ thông', 'Làm mướn', 'Nghiên cứu'],
    'Nghỉ hưu / Không làm việc': ['Không', 'Không có', 'Không có thông tin', 'Ko', 'Không ', 'Đã mất', 'Nội trợ', 'Nghỉ Hưu'],
    'Thủ công / May mặc': ['Thủ công', 'Thợ may', 'May gia công', 'Nghề may'],
    'Nghề khác': ['.', 'X', 'Điện công nghiệp', 'Kế toán trưởng cửa hàng', 'Kỹ sư hóa công nghiệp', 'Cấp dưỡng', 'Không có ', 'ko có ', 'Làm ', 'Làm ruỗng']
}

# Hàm để chuẩn hóa các nghề
def group_job(job_name):
    for group, keywords in job_groups.items():
        if any(keyword.lower() in str(job_name).strip().lower() for keyword in keywords):
            return group
    return 'Nghề khác'  # Trả về nhóm khác nếu không tìm thấy nhóm nào khớp

# Áp dụng hàm chuẩn hóa vào cột 'Cau_13a__Ten_cu_the_nghe_khac_'
df['Cau_13a__Ten_cu_the_nghe_khac_'] = df['Cau_13a__Ten_cu_the_nghe_khac_'].apply(lambda x: group_job(str(x) if pd.notnull(x) else ''))

In [39]:
# Định nghĩa các nhóm nghề chính
job_groups = {
    'Nông nghiệp': ['Kĩ thuật trồng cây', 'Nông nghiệp', 'Nông nghiêp', 'Trồng trọt', 'Nghề trồng trọt', 'Kĩ nông', 'Nghề làm vườn', 'Nghề trồng trọt, chăn nuôi', 'Làm vườn', 'Vườn ao chuồng', 'Làm muối'],
    'Điện': ['Điện dân dụng', 'Nghề điện', 'Nghề điện dân dụng', 'ĐIỆN DÂN DỤNG', 'Điện', 'điện', 'Điện dân dụng '],
    'May / Thêu': ['May', 'May, thêu', 'Lam may', 'Nghề may', 'Thêu', 'Nghề thêu', 'Thêu thùa', 'Nghề thêu vá', 'May ', 'Thêu thùa, đan lát', 'Đan móc'],
    'Tin học': ['Tin học', 'Tin học văn phòng', 'Tin học VP', 'Vi tính tin học', 'Tin học cơ bản', 'Môn nghề tin học', 'Công nghệ thông tin', 'Lập trình', 'Vi sinh dinh dưỡng'],
    'Nấu ăn': ['Nấu ăn', 'nấu ăn', 'Nấu ăn ', 'Nâu ăn', 'Dinh dưỡng', 'Nấu ăn, thêu thùa'],
    'Thủ công': ['Đan tre', 'Đan len', 'Đan rổ rá', 'Đan lưới', 'Đan lưới ', 'Nghề thủ công', 'Thủ công', 'Làm hoa giả', 'Sinh vật cảnh', 'Mĩ thuật'],
    'Giáo viên': ['Giáo viên', 'Giáo viên '],
    'Không nghề': ['Không', 'Không có', 'Ko', 'Kgoong', 'Khong', 'Không ', 'Không có '],
    'Nghề khác': ['Tâm lý con người', 'Tin học, cắm hoa, thêu, điện', 'Nhà báo', 'Makeup', 'Báo chí', 'Kí thuật nông nghiệp', 'Chế tạo máy', 'Tiếp thị', '.', 'X']
}

# Hàm để chuẩn hóa các nghề
def group_job(job_name):
    for group, keywords in job_groups.items():
        if any(keyword.lower() in str(job_name).strip().lower() for keyword in keywords):
            return group
    return 'Nghề khác'  # Trả về nhóm khác nếu không tìm thấy nhóm nào khớp

# Áp dụng hàm chuẩn hóa vào cột 'Cau_1b__Nghe_khac_'
df['Cau_1b__Nghe_khac_'] = df['Cau_1b__Nghe_khac_'].apply(lambda x: group_job(str(x) if pd.notnull(x) else ''))

In [40]:
place_groups_3a = {
    'Không có thông tin': ['Không có', 'Không', 'Không có ', 'không có', 'không', 'Không ', 'K có', 'Ko', 'O', 'K', 'Không cí', 'Khônh', 'ko có', 'Bt', 'A', 'ABC', 'Dfghj'],
    'Tại trường học': ['Tại trường', 'Tại trường THCS theo học', 'Tại trường THCS Bắc Sơn', 'Học tại trường THCS', 'Trường học ', 'Chỉ học tại trường'],
    'Làng nghề / Cơ sở sản xuất': ['Làng nghề', 'Làng nghề Sa Đéc - Đồng Tháp', 'Lang nghề', 'Làng nghề truyền thống ', 'Làng nghề ở địa phương', 
                                   'Xưởng may', 'Xưởng sản xuất đá vôi', 'Cơ sở sản xuất chế biến sản phẩm tư nhân', 
                                   'Cơ sở sản xuất nước mắm Cát hải', 'Cơ sở sản xuất may mặc', 'Cơ giới ninh bình'],
    'Trung tâm / Trại / Hợp tác xã': ['Trung tâm hướng nghiệp', 'Trung tâm', 'TT HN la san', 'TT nghề ', 'Trung tâm dạy nghê', 
                                      'Trung tâm dạy nghề ', 'Phòng nông nghiệp hợp tác xã', 'Trại chăn nuôi'],
    'Gia đình': ['Gia đình ', 'Tại gia', 'Nghỉ hè được bố mẹ đưa đến nơi bố mẹ công tác', 'Thực tế tại nhà dân của địa phương ', 'tại nhà'],
    'Nghề cụ thể': ['Thêu', 'May in ', 'Làm ruộng', 'Nhà hát, rạp chiếu phim', 'Sân vận động, trung tâm thể dục thể thao địa phương', 
                    'Cô giáo', 'Nơi thực tập sư phạm', 'Chỉ có đi lao động thôi'],
    'Khác': ['Đại học', 'Cty sản xuất']
}

# Hàm để chuẩn hóa các nghề
def group_job(job_name):
    for group, keywords in place_groups_3a.items():
        if any(keyword.lower() in str(job_name).strip().lower() for keyword in keywords):
            return group
    return 'Khác'  # Trả về nhóm khác nếu không tìm thấy nhóm nào khớp

# Áp dụng hàm chuẩn hóa vào cột 'Cau_3a___Ghi_ro_co_so_khac'
df['Cau_3a___Ghi_ro_co_so_khac'] = df['Cau_3a___Ghi_ro_co_so_khac'].apply(lambda x: group_job(str(x) if pd.notnull(x) else ''))

In [41]:
# Định nghĩa các nhóm chính
activity_groups_4a = {
    'Không có hoạt động': [str(x) for x in ['Không', 'Không ', 'Không có', 'Không có ', 'không', 'không có', 'Ko', 'ko có', 'Bt', 'K', 'Z', '..', 0, 'A', 'B', 'Ttt']],
    'Hoạt động thể thao': ['Thể thao'],
    'Hoạt động tư vấn / hướng nghiệp': [
        'Hướng nghiệp chọn trường đh phù hợp', 'Tư vấn nghề nghiệp', 'Tư vấn từ các doanh nghiệp', 
        'Tư vấn học đường', 'Tư vấn từ gia đình', 'Định hướng phát triển năng lực của bản thân ', 
        'Các trường đại học về giới thiệu'
    ],
    'Hoạt động tham quan / trải nghiệm': [
        'Hoạt động tham quan trải nghiệm', 'Tham quan một số cơ sở giáo dục ở địa phương', 
        'Tham gia sản xuất', 'Lao động vườn trường, lao động hè ', 'Tham gia sản xuất', 
        'Làm nông nghiệp', 'Lao động'
    ],
    'Hoạt động học tập / tự tìm hiểu': [
        'Tìm hiểu thông tin ngành nghề', 'Một số hoạt động đoàn thể, tự tìm hiểu.', 
        'Tự tìm hiểu để học nghề may', 'Đọc báo', 'mỗi học là vô được cấp 3 trước ', 
        'Lấy vdu minh hoạ'
    ],
    'Hoạt động tại trường': ['Tại trường', 'Trung tâm', 'Giáo dục nghề'],
    'Hoạt động gia đình': ['Gia đình'],
    'Khác': ['Giao lưu', 'Tập may']
}

# Hàm để chuẩn hóa các nghề
def group_job(job_name):
    for group, keywords in activity_groups_4a.items():
        if any(keyword.lower() in str(job_name).strip().lower() for keyword in keywords):
            return group
    return 'Khác'  # Trả về nhóm khác nếu không tìm thấy nhóm nào khớp

# Áp dụng hàm chuẩn hóa vào cột 'Cau_4a___Ghi_ro_hoat_dong_khac'
df['Cau_4a___Ghi_ro_hoat_dong_khac'] = df['Cau_4a___Ghi_ro_hoat_dong_khac'].apply(lambda x: group_job(str(x) if pd.notnull(x) else ''))

In [42]:
# Từ điển nhóm các giá trị
factors_groups_5a = {
    'Không có yếu tố khác': [
        'Không', 'Không ', 'Khong', 'Không có ', 'Không có', 'không có', 'không', 'ko có', 'Ko', 'Ko có', 'Z', 'K', 'Trt', 'O', 0, '.', 'A', 'B', 'Abc', 'Khônh'
    ],
    'Gia đình': [
        'Gia đình', 'Hoàn cảnh gia đình', 'Theo định hướng gia đình là chianh', 'Truyền thống gia đình', 'Từ gia đình', 'Giá Đình '
    ],
    'Sở thích cá nhân': [
        'Sở thích', 'Sở thích bản thân', 'sở thích theo giai đoạn', 'Bản thân yêu thích nghề mình chọn', 'Sở thích, sức khoẻ, năng lực'
    ],
    'Môi trường': [
        'Môi trường ', 'Môi trường làm việc Global toàn cầu/Nghiên cứu sinh du học', 'Văn hoá nơi làm việc'
    ],
    'Nhận thức bản thân': [
        'Nhận thức về thế giới xung quanh', 'Chủ quan của bản thân', 'Chưa hiểu rõ về năng lực của bản thân', 'Trưởng thành hơn', 
        'Khả năng ', 'Ý thức bản thân', 'Tâm lý, gia đình, sức khoẻ '
    ],
    'Khả năng tài chính': [
        'Khả năng tài chính gia đình.', 'Nhu cầu làm viêc', 'Nhu cầu của cuộc sống '
    ],
    'Tư vấn': [
        'Tư vấn của thầy cô giáo', 'Lựa chọn theo cảm tính vì không có sự tư vấn ', 'Theo bạn bè'
    ],
    'Không xác định': ['nan', None],
    'Yếu tố khác': [
        'Học vì mong muốn của phụ huynh', 'Theo truyền thông quảng cáo ', 'Hình tượng cụ thể của người đi trước mà mình biết', 
        'Sự coi trọng của xã hội về nghề lựa chọn ', 'Đi làm ', 'Kiếm thi nhập', 'Phù hợp sức khoẻ ', 'Thể thao', 'Tò mò', 'Ảnh hưởng', 
        'Phim, truyện','nan', None
    ]
}

# Hàm để chuẩn hóa các nghề
def group_job(job_name):
    for group, keywords in activity_groups_4a.items():
        if any(keyword.lower() in str(job_name).strip().lower() for keyword in keywords):
            return group
    return 'Khác'  # Trả về nhóm khác nếu không tìm thấy nhóm nào khớp

# Áp dụng hàm chuẩn hóa vào cột 'Cau_5a___Ghi_ro_yeu_to_khac'
df['Cau_5a___Ghi_ro_yeu_to_khac'] = df['Cau_5a___Ghi_ro_yeu_to_khac'].apply(lambda x: group_job(str(x) if pd.notnull(x) else ''))

In [43]:
from unidecode import unidecode
# Danh sách lý do đầy đủ
all_groups = [
    'Nhóm kỹ thuật', 'Nhóm nghệ thuật', 'Nhóm xã hội', 'Nhóm nghiên cứu',
    'Nhóm quản lý', 'Nhóm nghiệp vụ'
]

# Chuẩn hóa tên cột thành tiếng Việt không dấu và không có khoảng trắng
column_names = [
    f"Cau_7__b{unidecode(reason).replace(' ', '_').replace(',', '').lower()}"
    for reason in all_groups
]

# Tạo các cột nhị phân
for reason, col_name in zip(all_groups, column_names):
    df[col_name] = df['Cau_7__Khi_hoc_THCS__anh_chi_b'].apply(
        lambda x: 1 if pd.notna(x) and reason in x else 0
    )

# Tạo cột `cau_10a_____li_do_khac` và đặt giá trị mặc định là 0
if 'no_Cau_7__Khi_hoc_THCS__anh_chi_b' not in df.columns:
    df['no_Cau_7__Khi_hoc_THCS__anh_chi_b'] = 0

# Xử lý giá trị NaN: Nếu giá trị là NaN, điền 1 vào cột `cau_10a_____li_do_khac`
df['no_Cau_7__Khi_hoc_THCS__anh_chi_b'] = df.apply(
    lambda row: 1 if pd.isna(row['Cau_7__Khi_hoc_THCS__anh_chi_b']) else row['no_Cau_7__Khi_hoc_THCS__anh_chi_b'],
    axis=1
)

df = df.drop(columns=['Cau_7__Khi_hoc_THCS__anh_chi_b', 'no_Cau_7__Khi_hoc_THCS__anh_chi_b'], errors='ignore')

In [44]:
from unidecode import unidecode
import pandas as pd

# Danh sách lý do đầy đủ
all_groups = [
    'Nhóm kỹ thuật', 'Nhóm nghệ thuật', 'Nhóm xã hội', 'Nhóm nghiên cứu',
    'Nhóm quản lý', 'Nhóm nghiệp vụ'
]

# Vòng lặp qua các cột từ Cau_7__Khi_hoc_THCS__anh_chi_b_2 đến Cau_7__Khi_hoc_THCS__anh_chi_b_18
for i in range(2, 19):
    # Tên cột nguồn
    col_source = f"Cau_7__Khi_hoc_THCS__anh_chi_b_{i}"

    # Chuẩn hóa tên cột cho từng lý do
    column_names = [
        f"Cau_7__b_{i}_{unidecode(reason).replace(' ', '_').replace(',', '').lower()}"
        for reason in all_groups
    ]

    # Tạo các cột nhị phân cho từng lý do
    for reason, col_name in zip(all_groups, column_names):
        df[col_name] = df[col_source].apply(
            lambda x: 1 if pd.notna(x) and reason in str(x) else 0
        )

    # Tạo cột `no_Cau_7__Khi_hoc_THCS__anh_chi_b_{i}` để xử lý giá trị NaN
    col_na_name = f"no_Cau_7__Khi_hoc_THCS__anh_chi_b_{i}"
    df[col_na_name] = df[col_source].apply(
        lambda x: 1 if pd.isna(x) else 0
    )

    # Loại bỏ cột gốc sau khi xử lý
    df = df.drop(columns=[col_source, col_na_name], errors='ignore')


C:\Users\vongu\AppData\Local\Temp\ipykernel_16472\1628206331.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = df[col_source].apply(
C:\Users\vongu\AppData\Local\Temp\ipykernel_16472\1628206331.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = df[col_source].apply(
C:\Users\vongu\AppData\Local\Temp\ipykernel_16472\1628206331.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining 

In [45]:
# Tạo từ điển để chuẩn hóa dữ liệu
normalization_dict = {
    'Tương đối ảnh hưởng': 'Tương đối ảnh hưởng',
    'Hầu như không ảnh hưởng': 'Hầu như không ảnh hưởng',
    'Không ảnh hưởng': 'Không ảnh hưởng',
    'Ảnh hưởng đôi chút': 'Ảnh hưởng đôi chút',
    'Ảnh hưởng mạnh': 'Ảnh hưởng mạnh',
    'Tương đối ảnh hưởng': 'Tương đối ảnh hưởng',
    'Ảnh hưởng đôi chút': 'Ảnh hưởng đôi chút',
    'Ảnh hưởng mạnh': 'Ảnh hưởng mạnh'
}

# Áp dụng chuẩn hóa
df['Cau_5__Khi_hoc_THCS__anh_chi_t'] = df['Cau_5__Khi_hoc_THCS__anh_chi_t'].map(normalization_dict)


In [46]:
# Từ điển chuẩn hóa
normalization_dict = {
    'Tương đối ảnh hưởng': 'Tương đối ảnh hưởng',
    'Hầu như không ảnh hưởng': 'Hầu như không ảnh hưởng',
    'Không ảnh hưởng': 'Không ảnh hưởng',
    'Ảnh hưởng đôi chút': 'Ảnh hưởng đôi chút',
    'Ảnh hưởng mạnh': 'Ảnh hưởng mạnh',
    'Tương đối ảnh hưởng': 'Tương đối ảnh hưởng',
    'Ảnh hưởng đôi chút': 'Ảnh hưởng đôi chút',
    'Ảnh hưởng mạnh': 'Ảnh hưởng mạnh'
}

# Vòng lặp chuẩn hóa các cột từ 1 đến 18
for i in range(2, 22):
    column_name = f'Cau_5__Khi_hoc_THCS__anh_chi_t_{i}'  # Tên cột
    if column_name in df.columns:  # Kiểm tra cột có tồn tại không
        df[column_name] = df[column_name].map(normalization_dict)  # Chuẩn hóa
        print(f"Đã chuẩn hóa cột: {column_name}")  # In thông báo
    else:
        print(f"Cột {column_name} không tồn tại trong DataFrame")  # Thông báo nếu không có cột

# Kiểm tra lại các giá trị duy nhất sau khi chuẩn hóa
for i in range(2, 22):
    column_name = f'Cau_5__Khi_hoc_THCS__anh_chi_t_{i}'
    if column_name in df.columns:
        print(f"Giá trị duy nhất trong cột {column_name}: {df[column_name].unique()}")


Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_2
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_3
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_4
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_5
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_6
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_7
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_8
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_9
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_10
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_11
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_12
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_13
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_14
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_15
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_16
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_17
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_18
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_19
Đã chuẩn hóa cột: Cau_5__Khi_hoc_THCS__anh_chi_t_20
Đã chuẩn hóa cột: Ca

In [47]:
normalization_dict = {
    'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất…)': 
        'Kĩ thuật, công nghệ',
    'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất, nông nghiệp, thủ công…)': 
        'Kĩ thuật, công nghệ',
    'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất, nông nghiệp, thủ công…)': 
        'Kĩ thuật, công nghệ',
    'Nhóm nghề khác': 
        'Nhóm nghề khác',
    'Quản lý (tiếp thị, bán hàng, quản trị kinh doanh, cảnh sát, thanh tra,…)': 
        'Quản lý',
    'Xã hội (giáo viên, huấn luyện viên, hoạt động công tác xã hội…)': 
        'Xã hội',
    'Nghiệp vụ (như thư kí, kiểm toán, kế toán, hành chính văn phòng…)': 
        'Nghiệp vụ',
    'Nghiên cứu (như toán học, tâm lí học, xã hội học, Khoa học tự nhiên,…)': 
        'Nghiên cứu',
    'Nghệ thuật (viết văn, đạo diễn, dẫn chương trình, diễn viên, họa sĩ,…)': 
        'Nghệ thuật'
}

# Chuẩn hóa cột 'Cau_12________Nghe_nghiep_cua_'
column_name = 'Cau_12________Nghe_nghiep_cua_'
if column_name in df.columns:
    df[column_name] = df[column_name].map(normalization_dict)
    print(f"Đã chuẩn hóa cột: {column_name}")
else:
    print(f"Cột {column_name} không tồn tại trong DataFrame")

# Kiểm tra các giá trị duy nhất sau khi chuẩn hóa
if column_name in df.columns:
    print(f"Giá trị duy nhất trong cột {column_name} sau khi chuẩn hóa:")
    print(df[column_name].unique())

Đã chuẩn hóa cột: Cau_12________Nghe_nghiep_cua_
Giá trị duy nhất trong cột Cau_12________Nghe_nghiep_cua_ sau khi chuẩn hóa:
['Kĩ thuật, công nghệ' 'Nhóm nghề khác' 'Quản lý' 'Xã hội' 'Nghiệp vụ'
 'Nghiên cứu' 'Nghệ thuật']


In [48]:
# Từ điển chuẩn hóa giá trị
normalization_dict_cau_1a = {
    'Kĩ thuật, công nghệ (như điện, kĩ thuật nông nghiệp, xây dựng, cơ khí, điện tử, địa chất…)': 
        'Kĩ thuật, công nghệ',
    'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất, nông nghiệp, thủ công…)': 
        'Kĩ thuật, công nghệ',
    'Kĩ thuật, công nghệ (như kiến trúc, xây dựng, cơ khí, điện tử, địa chất, nông nghiệp, thủ công…)': 
        'Kĩ thuật, công nghệ',
    'Xã hội (giáo viên, huấn luyện viên, hoạt động công tác xã hội…)': 
        'Xã hội',
    'Nhóm nghề khác': 
        'Nhóm nghề khác',
    'Nghệ thuật (viết văn, đạo diễn, dẫn chương trình, diễn viên, họa sĩ,…)': 
        'Nghệ thuật',
    'Quản lý (tiếp thị, bán hàng, quản trị kinh doanh, cảnh sát, thanh tra,…)': 
        'Quản lý',
    'Nghiên cứu (như toán học, tâm lí học, xã hội học, Khoa học tự nhiên,…)': 
        'Nghiên cứu',
    'Nghiệp vụ (như thư kí, kiểm toán, kế toán, hành chính văn phòng…)': 
        'Nghiệp vụ',
}

# Tên cột cần chuẩn hóa
column_name_cau_1a = 'Cau_1a___Neu_co__anh_chi_duoc_'

# Chuẩn hóa cột
if column_name_cau_1a in df.columns:
    df[column_name_cau_1a] = df[column_name_cau_1a].map(normalization_dict_cau_1a)
    print(f"Đã chuẩn hóa cột: {column_name_cau_1a}")
else:
    print(f"Cột {column_name_cau_1a} không tồn tại trong DataFrame")

# Kiểm tra các giá trị duy nhất sau khi chuẩn hóa
if column_name_cau_1a in df.columns:
    print(f"Giá trị duy nhất trong cột {column_name_cau_1a} sau khi chuẩn hóa:")
    print(df[column_name_cau_1a].unique())


Đã chuẩn hóa cột: Cau_1a___Neu_co__anh_chi_duoc_
Giá trị duy nhất trong cột Cau_1a___Neu_co__anh_chi_duoc_ sau khi chuẩn hóa:
['Kĩ thuật, công nghệ' nan 'Xã hội' 'Nhóm nghề khác' 'Nghệ thuật'
 'Quản lý' 'Nghiên cứu' 'Nghiệp vụ']


In [49]:
display_column_info(df)

Column: Cau_2____Loai_hinh_truong_khi_
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_3____Loai_hinh_lop_khi_anh
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_5_____Gioi_tinh__
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_6_____Dan_toc__
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_7____Tuoi_cua_anh_chi__
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_8_____Sau_khi_hoc_xong_THC
  Data Type: object
  Non-null Count: 1067
  Total Count: 1067
  Missing Values Count: 0
  Missing Values Percentage: 0.00%



Column: Cau_8a_____Nghe_ngh

Column: Cau_1a___Neu_co__anh_chi_duoc_
  Data Type: object
  Non-null Count: 622
  Total Count: 1067
  Missing Values Count: 445
  Missing Values Percentage: 41.71%

  Column Cau_1a___Neu_co__anh_chi_duoc_ có 41.71% giá trị bị thiếu , cần xử lý

Column: Cau_1c__Anh_chi_dat_duoc_chung
  Data Type: object
  Non-null Count: 471
  Total Count: 1067
  Missing Values Count: 596
  Missing Values Percentage: 55.86%

  Column Cau_1c__Anh_chi_dat_duoc_chung có 55.86% giá trị bị thiếu , cần xử lý


Column: Cau_1d___Cong_viec_hien_nay_cu
  Data Type: object
  Non-null Count: 531
  Total Count: 1067
  Missing Values Count: 536
  Missing Values Percentage: 50.23%

  Column Cau_1d___Cong_viec_hien_nay_cu có 50.23% giá trị bị thiếu , cần xử lý


Column: Cau_1e____Trong_cuoc_song__anh
  Data Type: object
  Non-null Count: 533
  Total Count: 1067
  Missing Values Count: 534
  Missing Values Percentage: 50.05%

  Column Cau_1e____Trong_cuoc_song__anh có 50.05% giá trị bị thiếu , cần xử lý



In [50]:
column_name_cau_1a = 'Cau_1a___Neu_co__anh_chi_duoc_'
column_name_cau_1c = 'Cau_1c__Anh_chi_dat_duoc_chung'
column_name_cau_1d = 'Cau_1d___Cong_viec_hien_nay_cu'
column_name_cau_1e = 'Cau_1e____Trong_cuoc_song__anh'

# Kiểm tra xem cột có tồn tại không
if column_name_cau_1a in df.columns:
    # Thay thế NaN bằng 'Không học nghề'
    df[column_name_cau_1a] = df[column_name_cau_1a].fillna('Không học nghề')
    df[column_name_cau_1c] = df[column_name_cau_1c].fillna('Không học nghề')
    df[column_name_cau_1d] = df[column_name_cau_1d].fillna('Không học nghề')
    df[column_name_cau_1e] = df[column_name_cau_1e].fillna('Không học nghề')
    print(f"Đã thay thế giá trị NaN bằng 'Không học nghề'")
else:
    print(f"Cột {column_name_cau_1a} không tồn tại trong DataFrame")


Đã thay thế giá trị NaN bằng 'Không học nghề'


In [51]:
# Lọc danh sách các cột có giá trị NaN
columns_with_nan = df.columns[df.isna().any()].tolist()

# In ra danh sách các cột có giá trị NaN
if columns_with_nan:
    print("Các cột có giá trị NaN:")
    for col in columns_with_nan:
        nan_count = df[col].isna().sum()
        print(f"- {col}: {nan_count} giá trị NaN")
else:
    print("Không có cột nào chứa giá trị NaN.")


Các cột có giá trị NaN:
- Cau_5__Khi_hoc_THCS__anh_chi_t_2: 1 giá trị NaN
- Cau_5__Khi_hoc_THCS__anh_chi_t_3: 1 giá trị NaN
- Cau_5__Khi_hoc_THCS__anh_chi_t_4: 1 giá trị NaN
- Cau_5__Khi_hoc_THCS__anh_chi_t_5: 1 giá trị NaN
- Cau_5__Khi_hoc_THCS__anh_chi_t_6: 1 giá trị NaN
- Cau_5__Khi_hoc_THCS__anh_chi_t_7: 1 giá trị NaN
- Cau_5__Khi_hoc_THCS__anh_chi_t_11: 1 giá trị NaN
- Cau_5__Khi_hoc_THCS__anh_chi_t_12: 1 giá trị NaN
- Cau_5__Khi_hoc_THCS__anh_chi_t_17: 1 giá trị NaN


# chuẩn hóa

In [52]:
# In danh sách tên các cột
print(df.columns.tolist())

['Cau_2____Loai_hinh_truong_khi_', 'Cau_3____Loai_hinh_lop_khi_anh', 'Cau_5_____Gioi_tinh__', 'Cau_6_____Dan_toc__', 'Cau_7____Tuoi_cua_anh_chi__', 'Cau_8_____Sau_khi_hoc_xong_THC', 'Cau_8a_____Nghe_nghiep_cua_anh', 'Cau_9_____Nghe_nghiep_hien_tai', 'Cau_10______Anh_chi_da_tung_th', 'Cau_10b______Neu_Ly_do_khac_kh', 'Cau_10c_______Cong_viec_ma_anh', 'Cau_11_______Nghe_nghiep_ma_an', 'Cau_11a__Ten_cu_the_nghe_khac_', 'Cau_12________Nghe_nghiep_cua_', 'Cau_12a__Ten_cu_the_nghe_khac_', 'Cau_13________Nghe_nghiep_cua_', 'Cau_13a__Ten_cu_the_nghe_khac_', 'Cau_14________Diem_trung_binh_', 'Cau_15________Nam_hoc_lop_9_an', 'Cau_16________Nam_hoc_lop_9_an', 'Cau_17________Nam_hoc_lop_9_an', 'Cau_18________Nam_hoc_lop_9_an', 'Cau_1__Khi_hoc_THCS__anh_chi_c', 'Cau_1a___Neu_co__anh_chi_duoc_', 'Cau_1b__Nghe_khac_', 'Cau_1c__Anh_chi_dat_duoc_chung', 'Cau_1d___Cong_viec_hien_nay_cu', 'Cau_1e____Trong_cuoc_song__anh', 'Cau_2__Khi_hoc_THCS__anh_chi_d', 'Cau_2__Khi_hoc_THCS__anh_chi_d_2', 'Cau_2__Khi_

In [53]:
import os
import pandas as pd

def standardize_column(df, column_name, save_path="model/SIU/standardize"):
    """
    Chuẩn hóa các giá trị trong một cột của DataFrame và lưu mapping vào file CSV.

    Parameters:
    - df (pd.DataFrame): DataFrame chứa cột cần chuẩn hóa.
    - column_name (str): Tên cột cần chuẩn hóa.
    - save_path (str): Đường dẫn thư mục để lưu file mapping.

    Returns:
    - pd.DataFrame: DataFrame sau khi cột được chuẩn hóa.
    """
    # Tạo thư mục lưu trữ nếu chưa tồn tại
    os.makedirs(save_path, exist_ok=True)

    # Lấy các giá trị duy nhất và tạo mapping
    unique_values = df[column_name].dropna().unique()
    mapping = {value: idx for idx, value in enumerate(unique_values)}

    # Chuẩn hóa giá trị trong cột
    df[column_name] = df[column_name].map(mapping)

    # Xử lý giá trị NaN nếu có (chọn giá trị mặc định là -1)
    if df[column_name].isna().any():
        df[column_name].fillna(-1, inplace=True)

    # Lưu mapping vào file CSV
    mapping_df = pd.DataFrame(list(mapping.items()), columns=["Original Value", "Mapped Value"])
    file_path = os.path.join(save_path, f"{column_name}.csv")
    mapping_df.to_csv(file_path, index=False, encoding='utf-8')

    print(f"Mapping for column '{column_name}' saved to '{file_path}'")
    return df

# Áp dụng hàm cho tất cả các cột cần chuẩn hóa
columns_to_standardize = ['Cau_2____Loai_hinh_truong_khi_', 'Cau_3____Loai_hinh_lop_khi_anh', 'Cau_5_____Gioi_tinh__', 'Cau_6_____Dan_toc__', 'Cau_7____Tuoi_cua_anh_chi__', 'Cau_8_____Sau_khi_hoc_xong_THC', 'Cau_8a_____Nghe_nghiep_cua_anh', 'Cau_9_____Nghe_nghiep_hien_tai', 'Cau_10______Anh_chi_da_tung_th', 'Cau_10b______Neu_Ly_do_khac_kh', 'Cau_10c_______Cong_viec_ma_anh', 'Cau_11_______Nghe_nghiep_ma_an', 'Cau_11a__Ten_cu_the_nghe_khac_', 'Cau_12________Nghe_nghiep_cua_', 'Cau_12a__Ten_cu_the_nghe_khac_', 'Cau_13________Nghe_nghiep_cua_', 'Cau_13a__Ten_cu_the_nghe_khac_', 'Cau_14________Diem_trung_binh_', 'Cau_15________Nam_hoc_lop_9_an', 'Cau_16________Nam_hoc_lop_9_an', 'Cau_17________Nam_hoc_lop_9_an', 'Cau_18________Nam_hoc_lop_9_an', 'Cau_1__Khi_hoc_THCS__anh_chi_c', 'Cau_1a___Neu_co__anh_chi_duoc_', 'Cau_1b__Nghe_khac_', 'Cau_1c__Anh_chi_dat_duoc_chung', 'Cau_1d___Cong_viec_hien_nay_cu', 'Cau_1e____Trong_cuoc_song__anh', 'Cau_2__Khi_hoc_THCS__anh_chi_d', 'Cau_2__Khi_hoc_THCS__anh_chi_d_2', 'Cau_2__Khi_hoc_THCS__anh_chi_d_3', 'Cau_2__Khi_hoc_THCS__anh_chi_d_4', 'Cau_2__Khi_hoc_THCS__anh_chi_d_5', 'Cau_2__Khi_hoc_THCS__anh_chi_d_6', 'Cau_3___Khi_hoc_THCS__anh_chi_', 'Cau_3___Khi_hoc_THCS__anh_chi__2', 'Cau_3___Khi_hoc_THCS__anh_chi__3', 'Cau_3___Khi_hoc_THCS__anh_chi__4', 'Cau_3___Khi_hoc_THCS__anh_chi__5', 'Cau_3___Khi_hoc_THCS__anh_chi__6', 'Cau_3___Khi_hoc_THCS__anh_chi__7', 'Cau_3___Khi_hoc_THCS__anh_chi__8', 'Cau_3a___Ghi_ro_co_so_khac', 'Cau_4__Khi_hoc_THCS__anh_chi_d', 'Cau_4__Khi_hoc_THCS__anh_chi_d_2', 'Cau_4__Khi_hoc_THCS__anh_chi_d_3', 'Cau_4__Khi_hoc_THCS__anh_chi_d_4', 'Cau_4__Khi_hoc_THCS__anh_chi_d_5', 'Cau_4__Khi_hoc_THCS__anh_chi_d_6', 'Cau_4__Khi_hoc_THCS__anh_chi_d_7', 'Cau_4__Khi_hoc_THCS__anh_chi_d_8', 'Cau_4__Khi_hoc_THCS__anh_chi_d_9', 'Cau_4__Khi_hoc_THCS__anh_chi_d_10', 'Cau_4__Khi_hoc_THCS__anh_chi_d_11', 'Cau_4a___Ghi_ro_hoat_dong_khac', 'Cau_5__Khi_hoc_THCS__anh_chi_t', 'Cau_5__Khi_hoc_THCS__anh_chi_t_2', 'Cau_5__Khi_hoc_THCS__anh_chi_t_3', 'Cau_5__Khi_hoc_THCS__anh_chi_t_4', 'Cau_5__Khi_hoc_THCS__anh_chi_t_5', 'Cau_5__Khi_hoc_THCS__anh_chi_t_6', 'Cau_5__Khi_hoc_THCS__anh_chi_t_7', 'Cau_5__Khi_hoc_THCS__anh_chi_t_8', 'Cau_5__Khi_hoc_THCS__anh_chi_t_9', 'Cau_5__Khi_hoc_THCS__anh_chi_t_10', 'Cau_5__Khi_hoc_THCS__anh_chi_t_11', 'Cau_5__Khi_hoc_THCS__anh_chi_t_12', 'Cau_5__Khi_hoc_THCS__anh_chi_t_13', 'Cau_5__Khi_hoc_THCS__anh_chi_t_14', 'Cau_5__Khi_hoc_THCS__anh_chi_t_15', 'Cau_5__Khi_hoc_THCS__anh_chi_t_16', 'Cau_5__Khi_hoc_THCS__anh_chi_t_17', 'Cau_5__Khi_hoc_THCS__anh_chi_t_18', 'Cau_5__Khi_hoc_THCS__anh_chi_t_19', 'Cau_5__Khi_hoc_THCS__anh_chi_t_20', 'Cau_5__Khi_hoc_THCS__anh_chi_t_21', 'Cau_5a___Ghi_ro_yeu_to_khac', 'Cau_6__Khi_hoc_THCS__anh_chi_t', 'Cau_6__Khi_hoc_THCS__anh_chi_t_2', 'Cau_6__Khi_hoc_THCS__anh_chi_t_3', 'Cau_6__Khi_hoc_THCS__anh_chi_t_4', 'Cau_6__Khi_hoc_THCS__anh_chi_t_5', 'Cau_6__Khi_hoc_THCS__anh_chi_t_6', 'Cau_8__Khi_hoc_THCS__anh_chi_c', 'Cau_9__Khi_hoc_THCS__anh_chi_c', 'Cau_10__Hay_cho_biet_vi_sao_An', 'Cau_10__Hay_cho_biet_vi_sao_An_2', 'Cau_10__Hay_cho_biet_vi_sao_An_3', 'Cau_10__Hay_cho_biet_vi_sao_An_4', 'Cau_10__Hay_cho_biet_vi_sao_An_5', 'Cau_10__Hay_cho_biet_vi_sao_An_6', 'Cau_10__Hay_cho_biet_vi_sao_An_7', 'Cau_10__Hay_cho_biet_vi_sao_An_8', 'Cau_10__Hay_cho_biet_vi_sao_An_9', 'Cau_10__Hay_cho_biet_vi_sao_An_10', 'Cau_10__Hay_cho_biet_vi_sao_An_11', 'Cau_10__Hay_cho_biet_vi_sao_An_12', 'Cau_10__Hay_cho_biet_vi_sao_An_13', 'Cau_10__Hay_cho_biet_vi_sao_An_14', 'Cau_10__Hay_cho_biet_vi_sao_An_15', 'Cau_10__Hay_cho_biet_vi_sao_An_16', 'Cau_10__Hay_cho_biet_vi_sao_An_17', 'Cau_10__Hay_cho_biet_vi_sao_An_18', 'Cau_10__Hay_cho_biet_vi_sao_An_19', 'Cau_10__Hay_cho_biet_vi_sao_An_20', 'Cau_10__Hay_cho_biet_vi_sao_An_21', 'Cau_10__Hay_cho_biet_vi_sao_An_22', 'Cau_10__Hay_cho_biet_vi_sao_An_23', 'Cau_10__Hay_cho_biet_vi_sao_An_24', 'Cau_10__Hay_cho_biet_vi_sao_An_25', 'Cau_10__Hay_cho_biet_vi_sao_An_26', 'Cau_10__Hay_cho_biet_vi_sao_An_27', 'Cau_11__Hien_tai__Anh_chi_cam_', 'Cau_11__Hien_tai__Anh_chi_cam__2', 'Cau_11__Hien_tai__Anh_chi_cam__3', 'Cau_11__Hien_tai__Anh_chi_cam__4', 'Cau_12__Tich_vao_nhung_cau_tra', 'Cau_12__Tich_vao_nhung_cau_tra_2', 'Cau_12__Tich_vao_nhung_cau_tra_3', 'Cau_12__Tich_vao_nhung_cau_tra_4', 'Cau_12__Tich_vao_nhung_cau_tra_5', 'Cau_12__Tich_vao_nhung_cau_tra_6', 'Cau_12__Tich_vao_nhung_cau_tra_7', 'Cau_12__Tich_vao_nhung_cau_tra_8', 'Cau_12__Tich_vao_nhung_cau_tra_9', 'Cau_12__Tich_vao_nhung_cau_tra_10', 'Cau_12__Tich_vao_nhung_cau_tra_11', 'Cau_12__Tich_vao_nhung_cau_tra_12', 'Cau_12__Tich_vao_nhung_cau_tra_13', 'Cau_12__Tich_vao_nhung_cau_tra_14', 'Cau_12__Tich_vao_nhung_cau_tra_15']
for col in columns_to_standardize:
    df = standardize_column(df, col)


Mapping for column 'Cau_2____Loai_hinh_truong_khi_' saved to 'model/SIU/standardize\Cau_2____Loai_hinh_truong_khi_.csv'


Mapping for column 'Cau_3____Loai_hinh_lop_khi_anh' saved to 'model/SIU/standardize\Cau_3____Loai_hinh_lop_khi_anh.csv'
Mapping for column 'Cau_5_____Gioi_tinh__' saved to 'model/SIU/standardize\Cau_5_____Gioi_tinh__.csv'
Mapping for column 'Cau_6_____Dan_toc__' saved to 'model/SIU/standardize\Cau_6_____Dan_toc__.csv'
Mapping for column 'Cau_7____Tuoi_cua_anh_chi__' saved to 'model/SIU/standardize\Cau_7____Tuoi_cua_anh_chi__.csv'
Mapping for column 'Cau_8_____Sau_khi_hoc_xong_THC' saved to 'model/SIU/standardize\Cau_8_____Sau_khi_hoc_xong_THC.csv'
Mapping for column 'Cau_8a_____Nghe_nghiep_cua_anh' saved to 'model/SIU/standardize\Cau_8a_____Nghe_nghiep_cua_anh.csv'
Mapping for column 'Cau_9_____Nghe_nghiep_hien_tai' saved to 'model/SIU/standardize\Cau_9_____Nghe_nghiep_hien_tai.csv'
Mapping for column 'Cau_10______Anh_chi_da_tung_th' saved to 'model/SIU/standardize\Cau_10______Anh_chi_da_tung_th.csv'
Mapping for column 'Cau_10b______Neu_Ly_do_khac_kh' saved to 'model/SIU/standardize\C

C:\Users\vongu\AppData\Local\Temp\ipykernel_16472\1976354895.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(-1, inplace=True)
C:\Users\vongu\AppData\Local\Temp\ipykernel_16472\1976354895.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [54]:
df.to_csv('model/SIU/Standardize.csv', index=False)